In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import seaborn as sns
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from matplotlib.backends.backend_pdf import PdfPages 

from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import os
import gmplot
import gmaps
import time, datetime
%matplotlib inline

C:\Users\Cara\Anaconda3\envs\summer_project_windows_1_0\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#convert time columns to single output 
def time_from_line(line):
    """returns HHMMSS from line in data file"""
    if int(line['Hours'])<10:
        hour='0'+str(line['Hours'])
    else:
        hour=str(line['Hours'])
    if int(line['Minute'])<10:
        minute='0'+str(line['Minute'])
    else:
        minute=str(line['Minute'])
    if int(line['Second'])<10:
        second='0'+str(line['Second'])
    else:
        second=str(line['Second'])
    return hour+minute+second

In [3]:
#convert date columns to single output 
def date_from_line(line):
    """returns DDMMYYYY from line in data file"""
    if int(line['Day'])<10:
        day='0'+str(line['Day'])
    else:
        day=str(line['Day'])
    if int(line[1])<10:
        month='0'+str(line['Month'])
    else:
        month=str(line['Month'])
    return day+month+str(line['Year'])

In [4]:
#read file
print(os.getcwd())
#df=pd.read_csv('clean_00010001.csv') #new file
df=pd.read_csv('00010001_model_data.csv') 
df.head(1)

C:\Users\Cara\Documents\COMPSCI\1A_PRACTICUM


,Year,Month,Day,Hours,Minute,Second,WeekDay,Timestamp,JourneyPatternID,VehicleJourneyID,StopID,Temperature (C),Visibility (km),Wind Speed (km/h),Cumulative Time Taken
0,2012,11,6,8,59,2,1,1352192342,10001,5461,226,7,10,29.6,0


In [5]:
df.shape

(33820, 15)

# FEATURE DESCRIPTION

In [6]:
df.dtypes

Year                      int64
Month                     int64
Day                       int64
Hours                     int64
Minute                    int64
Second                    int64
WeekDay                   int64
Timestamp                 int64
JourneyPatternID          int64
VehicleJourneyID          int64
StopID                    int64
Temperature (C)           int64
Visibility (km)          object
Wind Speed (km/h)        object
Cumulative Time Taken     int64
dtype: object

In [7]:
df.head(3)

,Year,Month,Day,Hours,Minute,Second,WeekDay,Timestamp,JourneyPatternID,VehicleJourneyID,StopID,Temperature (C),Visibility (km),Wind Speed (km/h),Cumulative Time Taken
0,2012,11,6,8,59,2,1,1352192342,10001,5461,226,7,10,29.6,0
1,2012,11,6,9,4,41,1,1352192681,10001,5461,231,8,10,31.5,339
2,2012,11,6,9,6,22,1,1352192782,10001,5461,1641,8,10,31.5,440


In [8]:
# arr = [226, 228, 229, 227, 230, 231, 1641, 1642, 213, 214, 4432, 119, 44, 45, 46, 47, 48, 49, 50, 51, 52, 265, 271, 340, 350, 351, 352, 353, 354, 355, 356, 357, 390, 372, 373, 374, 375, 2804, 376, 377, 378, 380]

# df['StopSeq'] = 0
# for item, row in df.iterrows():
#     for i in arr:s
#         if row['StopID'] == arr[i]:
#             row['StopSeq'] = arr.index(int(i))
# print(df['StopSeq'])

In [9]:
continuous_cols = ['Timestamp', 'Cumulative Time Taken']
categ_cols = ['Year', 'Month', 'Day', 'Hours', 'Minute', 'Second', 'WeekDay', 'JourneyPatternID', 'VehicleJourneyID', 'StopID', 'Temperature (C)', 'Visibility (km)', 'Wind Speed (km/h)']

In [10]:
for col_name in df:
    print(col_name, "\n", df[col_name].unique(), "\n", "number of unique values:", len(df[col_name].unique()), "\n", df[col_name].value_counts(), "\n")

Year 
 [2012 2013] 
 number of unique values: 2 
 2013    17951
2012    15869
Name: Year, dtype: int64 

Month 
 [11  1] 
 number of unique values: 2 
 1     17951
11    15869
Name: Month, dtype: int64 

Day 
 [ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
  1  2  3  4  5 31] 
 number of unique values: 31 
 8     1471
14    1461
15    1458
22    1453
7     1416
26    1383
29    1381
23    1360
9     1328
19    1320
16    1296
28    1296
21    1281
10    1248
17    1229
12    1185
6     1148
13    1137
24    1102
27    1082
20    1060
25     992
18     983
11     941
30     815
3      729
4      721
2      605
5      506
1      284
31     149
Name: Day, dtype: int64 

Hours 
 [ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23  7] 
 number of unique values: 17 
 14    3009
12    3007
13    2949
11    2948
10    2810
16    2743
15    2727
17    2627
18    2589
9     2111
19    1793
20    1521
21    1261
22    1031
23     563
8      130
7        1
Name: Hours, dt

In [11]:
dfcontDesc = df[continuous_cols].describe()
dfcontDesc

,Timestamp,Cumulative Time Taken
count,3.382000e+04,33820.000000
mean,1.355928e+09,1445.880012
std,2.638506e+06,904.203237
min,1.352192e+09,0.000000
25%,1.353316e+09,702.000000
50%,1.357213e+09,1418.000000
75%,1.358416e+09,2199.000000
max,1.359633e+09,4544.000000


In [84]:
dfcategDesc = df[categ_cols].describe(include='all')
dfcategDesc

,Year,Month,Day,Hours,Minute,Second,WeekDay,JourneyPatternID,VehicleJourneyID,StopID,Temperature (C),Visibility (km),Wind Speed (km/h)
count,33820.000000,33820.000000,33820.000000,33820.000000,33820.000000,33820.000000,33820.000000,33820.0,33820.000000,33820.000000,33820.000000,33820,33820
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,48
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,24.1
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27670,2406
mean,2012.530781,5.692194,16.598078,14.777469,28.589533,27.455322,2.664104,10001.0,7206.866351,365.972590,6.797694,NaN,NaN
std,0.499059,4.990590,8.011508,3.748713,17.375676,16.897224,1.830370,0.0,24594.723395,565.198169,3.727865,NaN,NaN
min,2012.000000,1.000000,1.000000,7.000000,0.000000,0.000000,0.000000,10001.0,4608.000000,44.000000,-5.000000,NaN,NaN
25%,2012.000000,1.000000,10.000000,12.000000,14.000000,10.000000,1.000000,10001.0,4669.000000,214.000000,4.000000,NaN,NaN
50%,2013.000000,1.000000,16.000000,14.000000,29.000000,27.000000,3.000000,10001.0,4812.000000,340.000000,7.000000,NaN,NaN
75%,2013.000000,11.000000,23.000000,18.000000,43.000000,44.000000,4.000000,10001.0,5496.500000,356.000000,10.000000,NaN,NaN


In [85]:
dfStop = df.loc[df["StopID"] == 226]
dfStop.head(3)

,Year,Month,Day,Hours,Minute,Second,WeekDay,Timestamp,JourneyPatternID,VehicleJourneyID,StopID,Temperature (C),Visibility (km),Wind Speed (km/h),Cumulative Time Taken
0,2012,11,6,8,59,2,1,1352192342,10001,5461,226,7,10,29.6,0
16,2012,11,6,9,21,8,1,1352193668,10001,5494,226,7,10,27.8,0
66,2012,11,6,10,0,9,1,1352196009,10001,5444,226,8,10,33.3,0


In [88]:
dfStop = df.loc[df["Month"] == 11]

In [89]:
x =dfStop
for col_name in x:
    print(col_name, "\n", x[col_name].unique(), "\n", "number of unique values:", len(x[col_name].unique()), "\n", x[col_name].value_counts(), "\n")

Year 
 [2012] 
 number of unique values: 1 
 2012    15869
Name: Year, dtype: int64 

Month 
 [11] 
 number of unique values: 1 
 11    15869
Name: Month, dtype: int64 

Day 
 [ 6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30] 
 number of unique values: 25 
 15    752
6     751
7     744
8     743
9     743
14    738
13    729
23    721
22    714
19    708
16    706
26    705
12    701
20    700
27    698
29    690
17    622
28    604
21    591
10    586
30    526
24    448
18    345
25    316
11    288
Name: Day, dtype: int64 

Hours 
 [ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23] 
 number of unique values: 16 
 12    1458
11    1454
10    1433
14    1401
13    1376
16    1255
15    1224
17    1188
18    1164
9     1030
19     848
20     698
21     589
22     434
23     258
8       59
Name: Hours, dtype: int64 

Minute 
 [59  4  6  7 15 18 19 22 24 29 30 33 34 35 36 50 21 23 26 27 28 38 39 43 44
 46 49 53 55 58  0  1  5  8 41 42 45 48 51 54 56 57 11 12 14

In [90]:
cols = ["Month", "Day", "WeekDay", "VehicleJourneyID"]
df3 = dfStop[cols]

In [91]:
all_dates=[] #list of all dates
for index, row in df.iterrows():
    date=date_from_line(row)
#     df['DateColumn']=date jan 31 for all lines of data??
    if date not in all_dates: # list of all dates
        all_dates.append(date)  
print(all_dates)

['06112012', '07112012', '08112012', '09112012', '10112012', '11112012', '12112012', '13112012', '14112012', '15112012', '16112012', '17112012', '18112012', '19112012', '20112012', '21112012', '22112012', '23112012', '24112012', '25112012', '26112012', '27112012', '28112012', '29112012', '30112012', '01012013', '02012013', '03012013', '04012013', '05012013', '06012013', '07012013', '08012013', '09012013', '10012013', '11012013', '12012013', '13012013', '14012013', '15012013', '16012013', '17012013', '18012013', '19012013', '20012013', '21012013', '22012013', '23012013', '24012013', '25012013', '26012013', '27012013', '28012013', '29012013', '30012013', '31012013']


key = date, value = (vjid, frequency)

{'01012013': [(15594, 1),
  (15622, 1),
  (15581, 1),
  (15609, 1),
  (15596, 1),
  (15624, 1),
  (15583, 1),
  (15611, 1),
  (15598, 1),
  (15626, 1),
  (15585, 1),
  (15613, 1),
  (15600, 1),
  (15628, 1),
  (15587, 1),
  (15615, 1),
  (15602, 1),
  (15630, 1),
  (15589, 1),
  (15617, 1),
  (15604, 1),
  (15632, 1),
  (15591, 1),
  (15619, 1),
  (15606, 1)],
 '02012013': [(4644, 1),
  (4677, 1),
  (4608, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4683, 1),
  (4633, 1),
  (4614, 1),
  (4671, 1),
  (4652, 1),
  (4685, 1),
  (4703, 1),
  (4635, 1),
  (4616, 1),
  (4673, 1),
  (4654, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '03012013': [(4644, 1),
  (4677, 1),
  (4695, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4650, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4614, 1),
  (4671, 1),
  (4652, 1),
  (4685, 1),
  (4703, 1),
  (4616, 1),
  (4673, 1),
  (4654, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '04012013': [(4644, 1),
  (4677, 1),
  (4695, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4650, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4614, 1),
  (4671, 1),
  (4652, 1),
  (4685, 1),
  (4635, 1),
  (4616, 1),
  (4673, 1),
  (4654, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '05012013': [(11262, 1),
  (11247, 1),
  (11273, 1),
  (11232, 1),
  (11264, 1),
  (11249, 1),
  (11275, 1),
  (11294, 1),
  (11266, 1),
  (11251, 1),
  (11283, 1),
  (11277, 1),
  (11236, 1),
  (11296, 1),
  (11268, 1),
  (11253, 1),
  (11285, 1),
  (11279, 1),
  (11298, 1),
  (11270, 1),
  (11287, 1),
  (11281, 1),
  (11300, 1),
  (11257, 1),
  (11289, 1),
  (11242, 1),
  (11302, 1),
  (11259, 1),
  (11291, 1),
  (11244, 1),
  (11304, 1),
  (11261, 1)],
 '06012013': [(15594, 1),
  (15622, 1),
  (15581, 1),
  (15609, 1),
  (15596, 1),
  (15624, 1),
  (15583, 1),
  (15611, 1),
  (15598, 1),
  (15626, 1),
  (15585, 1),
  (15613, 1),
  (15600, 1),
  (15628, 1),
  (15587, 1),
  (15615, 1),
  (15602, 1),
  (15630, 1),
  (15589, 1),
  (15617, 1),
  (15604, 1),
  (15632, 1),
  (15591, 1),
  (15619, 1),
  (15606, 1)],
 '06112012': [(5461, 1),
  (5494, 1),
  (5444, 1),
  (5425, 1),
  (5482, 1),
  (5463, 1),
  (5496, 1),
  (5514, 1),
  (5446, 1),
  (5427, 1),
  (5484, 1),
  (5465, 1),
  (5498, 1),
  (5516, 1),
  (5486, 1),
  (5467, 1),
  (5500, 1),
  (5518, 1),
  (5450, 1),
  (5488, 1),
  (5469, 1),
  (5502, 1),
  (5452, 1),
  (5433, 1),
  (5490, 1),
  (5471, 1),
  (5504, 1),
  (5454, 1),
  (5435, 1),
  (5473, 1),
  (5506, 1),
  (5456, 1),
  (5437, 1),
  (5475, 1)],
 '07012013': [(4644, 1),
  (4677, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4650, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4614, 1),
  (4671, 1),
  (4652, 1),
  (4685, 1),
  (4703, 1),
  (4616, 1),
  (4673, 1),
  (4654, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '07112012': [(5494, 1),
  (5512, 1),
  (5444, 1),
  (5425, 1),
  (5482, 1),
  (5463, 1),
  (5496, 1),
  (5514, 1),
  (5446, 1),
  (5427, 1),
  (5484, 1),
  (5465, 1),
  (5498, 1),
  (5516, 1),
  (5448, 1),
  (5486, 1),
  (5467, 1),
  (5500, 1),
  (5518, 1),
  (5450, 1),
  (5431, 1),
  (5488, 1),
  (5469, 1),
  (5502, 1),
  (5490, 1),
  (5471, 1),
  (5504, 1),
  (5454, 1),
  (5435, 1),
  (5473, 1),
  (5506, 1),
  (5456, 1),
  (5437, 1),
  (5475, 1)],
 '08012013': [(4644, 1),
  (4677, 1),
  (4695, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4614, 1),
  (4671, 1),
  (4652, 1),
  (4685, 1),
  (4703, 1),
  (4635, 1),
  (4616, 1),
  (4673, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '08112012': [(5461, 1),
  (5494, 1),
  (5512, 1),
  (5444, 1),
  (5425, 1),
  (5482, 1),
  (5463, 1),
  (5496, 1),
  (5514, 1),
  (5446, 1),
  (5427, 1),
  (5484, 1),
  (5465, 1),
  (5498, 1),
  (5516, 1),
  (5448, 1),
  (5486, 1),
  (5467, 1),
  (5500, 1),
  (5518, 1),
  (5450, 1),
  (5431, 1),
  (5488, 1),
  (5469, 1),
  (5502, 1),
  (5520, 1),
  (5452, 1),
  (5490, 1),
  (5504, 1),
  (5454, 1),
  (5435, 1),
  (5473, 1),
  (5506, 1),
  (5456, 1),
  (5437, 1),
  (5475, 1)],
 '09012013': [(4677, 1),
  (4695, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4679, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4614, 1),
  (4671, 1),
  (4685, 1),
  (4635, 1),
  (4616, 1),
  (4673, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '09112012': [(5461, 1),
  (5512, 1),
  (5444, 1),
  (5425, 1),
  (5482, 1),
  (5463, 1),
  (5496, 1),
  (5514, 1),
  (5446, 1),
  (5427, 1),
  (5484, 1),
  (5465, 1),
  (5498, 1),
  (5516, 1),
  (5448, 1),
  (5429, 1),
  (5486, 1),
  (5467, 1),
  (5450, 1),
  (5469, 1),
  (5452, 1),
  (5454, 1),
  (5435, 1),
  (5473, 1),
  (5506, 1),
  (5456, 1),
  (5437, 1),
  (5475, 1)],
 '10012013': [(4644, 1),
  (4677, 1),
  (4695, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4667, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4669, 1),
  (4650, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4671, 1),
  (4652, 1),
  (4685, 1),
  (4635, 1),
  (4673, 1),
  (4654, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '10112012': [(12138, 1),
  (12164, 1),
  (12123, 1),
  (12155, 1),
  (12140, 1),
  (12166, 1),
  (12125, 1),
  (12185, 1),
  (12157, 1),
  (12127, 1),
  (12176, 1),
  (12170, 1),
  (12129, 1),
  (12189, 1),
  (12161, 1),
  (12146, 1),
  (12172, 1),
  (12131, 1),
  (12191, 1),
  (12148, 1),
  (12180, 1),
  (12133, 1),
  (12193, 1),
  (12150, 1),
  (12182, 1),
  (12135, 1),
  (12195, 1),
  (12152, 1)],
 '11012013': [(4644, 1),
  (4677, 1),
  (4695, 1),
  (4627, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4629, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4614, 1),
  (4652, 1),
  (4685, 1),
  (4616, 1),
  (4654, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '11112012': [(16367, 1),
  (16354, 1),
  (16382, 1),
  (16369, 1),
  (16356, 1),
  (16384, 1),
  (16371, 1),
  (16358, 1),
  (16386, 1),
  (16373, 1),
  (16360, 1),
  (16388, 1),
  (16375, 1),
  (16362, 1),
  (16390, 1),
  (16377, 1),
  (16364, 1),
  (16392, 1),
  (16379, 1)],
 '12012013': [(11230, 1),
  (11273, 1),
  (11232, 1),
  (11264, 1),
  (11249, 1),
  (11275, 1),
  (11234, 1),
  (11294, 1),
  (11266, 1),
  (11251, 1),
  (11283, 1),
  (11277, 1),
  (11236, 1),
  (11296, 1),
  (11253, 1),
  (11285, 1),
  (11279, 1),
  (11298, 1),
  (11287, 1),
  (11281, 1),
  (11240, 1),
  (11300, 1),
  (11257, 1),
  (11289, 1),
  (11242, 1),
  (11302, 1),
  (11259, 1),
  (11291, 1),
  (11244, 1),
  (11304, 1),
  (11261, 1)],
 '12112012': [(5461, 1),
  (5494, 1),
  (5444, 1),
  (5425, 1),
  (5482, 1),
  (5463, 1),
  (5496, 1),
  (5514, 1),
  (5446, 1),
  (5427, 1),
  (5484, 1),
  (5465, 1),
  (5498, 1),
  (5516, 1),
  (5448, 1),
  (5429, 1),
  (5486, 1),
  (5467, 1),
  (5518, 1),
  (5450, 1),
  (5431, 1),
  (5488, 1),
  (5502, 1),
  (5520, 1),
  (5433, 1),
  (5490, 1),
  (5504, 1),
  (5454, 1),
  (5435, 1),
  (5473, 1),
  (5506, 1),
  (5456, 1),
  (5437, 1),
  (5475, 1)],
 '13012013': [(15594, 1),
  (15622, 1),
  (15581, 1),
  (15609, 1),
  (15596, 1),
  (15624, 1),
  (15583, 1),
  (15598, 1),
  (15626, 1),
  (15585, 1),
  (15613, 1),
  (15600, 1),
  (15628, 1),
  (15587, 1),
  (15602, 1),
  (15630, 1),
  (15589, 1),
  (15617, 1),
  (15604, 1),
  (15632, 1),
  (15591, 1),
  (15619, 1),
  (15606, 1)],
 '13112012': [(5461, 1),
  (5494, 1),
  (5512, 1),
  (5444, 1),
  (5425, 1),
  (5482, 1),
  (5496, 1),
  (5514, 1),
  (5446, 1),
  (5427, 1),
  (5484, 1),
  (5465, 1),
  (5498, 1),
  (5516, 1),
  (5448, 1),
  (5429, 1),
  (5486, 1),
  (5467, 1),
  (5500, 1),
  (5518, 1),
  (5450, 1),
  (5431, 1),
  (5488, 1),
  (5469, 1),
  (5502, 1),
  (5520, 1),
  (5452, 1),
  (5504, 1),
  (5454, 1),
  (5435, 1),
  (5473, 1),
  (5506, 1),
  (5456, 1),
  (5437, 1),
  (5475, 1)],
 '14012013': [(4644, 1),
  (4677, 1),
  (4695, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4650, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4614, 1),
  (4671, 1),
  (4652, 1),
  (4703, 1),
  (4616, 1),
  (4673, 1),
  (4654, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '14112012': [(5461, 1),
  (5494, 1),
  (5512, 1),
  (5444, 1),
  (5425, 1),
  (5482, 1),
  (5463, 1),
  (5496, 1),
  (5514, 1),
  (5446, 1),
  (5427, 1),
  (5484, 1),
  (5498, 1),
  (5516, 1),
  (5448, 1),
  (5429, 1),
  (5486, 1),
  (5467, 1),
  (5500, 1),
  (5450, 1),
  (5431, 1),
  (5488, 1),
  (5520, 1),
  (5504, 1),
  (5454, 1),
  (5435, 1),
  (5506, 1),
  (5456, 1),
  (5437, 1),
  (5475, 1)],
 '15012013': [(4644, 1),
  (4677, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4701, 1),
  (4633, 1),
  (4614, 1),
  (4671, 1),
  (4652, 1),
  (4703, 1),
  (4635, 1),
  (4673, 1),
  (4654, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '15112012': [(5461, 1),
  (5494, 1),
  (5444, 1),
  (5425, 1),
  (5482, 1),
  (5496, 1),
  (5514, 1),
  (5446, 1),
  (5427, 1),
  (5484, 1),
  (5465, 1),
  (5498, 1),
  (5516, 1),
  (5448, 1),
  (5486, 1),
  (5467, 1),
  (5500, 1),
  (5518, 1),
  (5450, 1),
  (5502, 1),
  (5520, 1),
  (5490, 1),
  (5471, 1),
  (5504, 1),
  (5454, 1),
  (5473, 1),
  (5506, 1),
  (5456, 1),
  (5437, 1),
  (5475, 1)],
 '16012013': [(4677, 1),
  (4695, 1),
  (4627, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4669, 1),
  (4650, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4685, 1),
  (4635, 1),
  (4673, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '16112012': [(5475, 1),
  (5461, 1),
  (5494, 1),
  (5512, 1),
  (5444, 1),
  (5425, 1),
  (5482, 1),
  (5463, 1),
  (5496, 1),
  (5514, 1),
  (5446, 1),
  (5427, 1),
  (5484, 1),
  (5465, 1),
  (5498, 1),
  (5516, 1),
  (5448, 1),
  (5429, 1),
  (5486, 1),
  (5518, 1),
  (5431, 1),
  (5488, 1),
  (5469, 1),
  (5520, 1),
  (5433, 1),
  (5471, 1),
  (5504, 1),
  (5454, 1),
  (5435, 1),
  (5473, 1),
  (5506, 1),
  (5456, 1),
  (5437, 1)],
 '17012013': [(4644, 1),
  (4677, 1),
  (4695, 1),
  (4627, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4669, 1),
  (4683, 1),
  (4633, 1),
  (4671, 1),
  (4685, 1),
  (4616, 1),
  (4673, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '17112012': [(12153, 1),
  (12138, 1),
  (12164, 1),
  (12123, 1),
  (12155, 1),
  (12140, 1),
  (12166, 1),
  (12125, 1),
  (12185, 1),
  (12157, 1),
  (12142, 1),
  (12174, 1),
  (12168, 1),
  (12127, 1),
  (12187, 1),
  (12159, 1),
  (12176, 1),
  (12170, 1),
  (12129, 1),
  (12189, 1),
  (12161, 1),
  (12146, 1),
  (12178, 1),
  (12172, 1),
  (12131, 1),
  (12191, 1),
  (12148, 1),
  (12180, 1),
  (12133, 1),
  (12193, 1),
  (12150, 1),
  (12182, 1),
  (12135, 1),
  (12195, 1),
  (12152, 1)],
 '18012013': [(4644, 1),
  (4677, 1),
  (4627, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4669, 1),
  (4650, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4614, 1),
  (4671, 1),
  (4703, 1),
  (4635, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '18112012': [(15672, 1),
  (15659, 1),
  (15687, 1),
  (15646, 1),
  (15674, 1),
  (15661, 1),
  (15689, 1),
  (15648, 1),
  (15676, 1),
  (15663, 1),
  (15691, 1),
  (15650, 1),
  (15678, 1),
  (15665, 1),
  (15693, 1),
  (15652, 1),
  (15680, 1),
  (15667, 1),
  (15695, 1),
  (15654, 1),
  (15682, 1),
  (15669, 1),
  (15697, 1),
  (15656, 1),
  (15684, 1),
  (15671, 1)],
 '19012013': [(11230, 1),
  (11262, 1),
  (11247, 1),
  (11273, 1),
  (11232, 1),
  (11264, 1),
  (11249, 1),
  (11275, 1),
  (11234, 1),
  (11294, 1),
  (11266, 1),
  (11251, 1),
  (11283, 1),
  (11277, 1),
  (11296, 1),
  (11268, 1),
  (11253, 1),
  (11285, 1),
  (11279, 1),
  (11238, 1),
  (11270, 1),
  (11255, 1),
  (11287, 1),
  (11240, 1),
  (11300, 1),
  (11257, 1),
  (11289, 1),
  (11242, 1),
  (11302, 1),
  (11259, 1),
  (11291, 1),
  (11244, 1),
  (11304, 1),
  (11261, 1)],
 '19112012': [(4784, 1),
  (4817, 1),
  (4835, 1),
  (4748, 1),
  (4805, 1),
  (4786, 1),
  (4819, 1),
  (4837, 1),
  (4769, 1),
  (4750, 1),
  (4807, 1),
  (4821, 1),
  (4771, 1),
  (4752, 1),
  (4809, 1),
  (4790, 1),
  (4823, 1),
  (4841, 1),
  (4754, 1),
  (4811, 1),
  (4825, 1),
  (4775, 1),
  (4756, 1),
  (4794, 1),
  (4827, 1),
  (4777, 1),
  (4796, 1),
  (4829, 1),
  (4779, 1),
  (4760, 1),
  (4798, 1)],
 '20012013': [(15607, 1),
  (15594, 1),
  (15622, 1),
  (15581, 1),
  (15609, 1),
  (15596, 1),
  (15624, 1),
  (15583, 1),
  (15611, 1),
  (15598, 1),
  (15626, 1),
  (15585, 1),
  (15613, 1),
  (15600, 1),
  (15587, 1),
  (15615, 1),
  (15602, 1),
  (15630, 1),
  (15589, 1),
  (15617, 1),
  (15604, 1),
  (15632, 1),
  (15591, 1),
  (15619, 1),
  (15606, 1)],
 '20112012': [(4784, 1),
  (4817, 1),
  (4835, 1),
  (4748, 1),
  (4805, 1),
  (4786, 1),
  (4819, 1),
  (4837, 1),
  (4769, 1),
  (4750, 1),
  (4807, 1),
  (4821, 1),
  (4839, 1),
  (4771, 1),
  (4752, 1),
  (4809, 1),
  (4790, 1),
  (4823, 1),
  (4841, 1),
  (4754, 1),
  (4811, 1),
  (4825, 1),
  (4756, 1),
  (4813, 1),
  (4794, 1),
  (4827, 1),
  (4777, 1),
  (4758, 1),
  (4796, 1),
  (4829, 1),
  (4779, 1),
  (4760, 1),
  (4798, 1)],
 '21012013': [(4644, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4650, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4614, 1),
  (4671, 1),
  (4652, 1),
  (4685, 1),
  (4703, 1),
  (4635, 1),
  (4673, 1),
  (4654, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '21112012': [(4784, 1),
  (4817, 1),
  (4835, 1),
  (4767, 1),
  (4748, 1),
  (4805, 1),
  (4786, 1),
  (4819, 1),
  (4837, 1),
  (4769, 1),
  (4750, 1),
  (4807, 1),
  (4788, 1),
  (4839, 1),
  (4771, 1),
  (4752, 1),
  (4809, 1),
  (4823, 1),
  (4841, 1),
  (4773, 1),
  (4754, 1),
  (4811, 1),
  (4825, 1),
  (4843, 1),
  (4775, 1),
  (4756, 1),
  (4813, 1),
  (4827, 1),
  (4777, 1),
  (4758, 1),
  (4796, 1),
  (4829, 1),
  (4779, 1),
  (4760, 1),
  (4798, 1)],
 '22012013': [(4644, 1),
  (4677, 1),
  (4695, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4650, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4671, 1),
  (4652, 1),
  (4685, 1),
  (4703, 1),
  (4616, 1),
  (4673, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '22112012': [(4784, 1),
  (4817, 1),
  (4835, 1),
  (4767, 1),
  (4748, 1),
  (4805, 1),
  (4786, 1),
  (4819, 1),
  (4837, 1),
  (4769, 1),
  (4750, 1),
  (4807, 1),
  (4788, 1),
  (4821, 1),
  (4839, 1),
  (4771, 1),
  (4752, 1),
  (4809, 1),
  (4790, 1),
  (4823, 1),
  (4841, 1),
  (4773, 1),
  (4811, 1),
  (4825, 1),
  (4813, 1),
  (4794, 1),
  (4777, 1),
  (4758, 1),
  (4796, 1),
  (4829, 1),
  (4779, 1),
  (4760, 1),
  (4798, 1)],
 '23012013': [(4644, 1),
  (4677, 1),
  (4695, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4650, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4614, 1),
  (4652, 1),
  (4685, 1),
  (4703, 1),
  (4616, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '23112012': [(4784, 1),
  (4817, 1),
  (4835, 1),
  (4767, 1),
  (4748, 1),
  (4805, 1),
  (4819, 1),
  (4837, 1),
  (4769, 1),
  (4750, 1),
  (4807, 1),
  (4788, 1),
  (4821, 1),
  (4839, 1),
  (4771, 1),
  (4752, 1),
  (4809, 1),
  (4825, 1),
  (4813, 1),
  (4794, 1),
  (4827, 1),
  (4777, 1),
  (4758, 1),
  (4796, 1),
  (4829, 1),
  (4779, 1),
  (4760, 1),
  (4798, 1)],
 '24012013': [(4644, 1),
  (4677, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4701, 1),
  (4633, 1),
  (4671, 1),
  (4635, 1),
  (4616, 1),
  (4673, 1),
  (4687, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '24112012': [(11433, 1),
  (11418, 1),
  (11444, 1),
  (11403, 1),
  (11435, 1),
  (11420, 1),
  (11446, 1),
  (11405, 1),
  (11465, 1),
  (11437, 1),
  (11422, 1),
  (11454, 1),
  (11448, 1),
  (11407, 1),
  (11467, 1),
  (11456, 1),
  (11450, 1),
  (11409, 1),
  (11469, 1),
  (11441, 1),
  (11458, 1),
  (11452, 1),
  (11411, 1),
  (11428, 1),
  (11460, 1),
  (11473, 1),
  (11430, 1),
  (11415, 1),
  (11475, 1),
  (11432, 1)],
 '25012013': [(4695, 1),
  (4627, 1),
  (4608, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4612, 1),
  (4650, 1),
  (4683, 1),
  (4614, 1),
  (4671, 1),
  (4652, 1),
  (4685, 1),
  (4703, 1),
  (4635, 1),
  (4616, 1),
  (4673, 1),
  (4687, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '25112012': [(15732, 1),
  (15760, 1),
  (15719, 1),
  (15747, 1),
  (15734, 1),
  (15762, 1),
  (15749, 1),
  (15736, 1),
  (15764, 1),
  (15751, 1),
  (15738, 1),
  (15725, 1),
  (15768, 1),
  (15727, 1),
  (15755, 1),
  (15742, 1),
  (15770, 1),
  (15729, 1),
  (15757, 1),
  (15744, 1)],
 '26012013': [(11262, 1),
  (11247, 1),
  (11273, 1),
  (11232, 1),
  (11264, 1),
  (11249, 1),
  (11275, 1),
  (11234, 1),
  (11266, 1),
  (11251, 1),
  (11283, 1),
  (11277, 1),
  (11236, 1),
  (11296, 1),
  (11253, 1),
  (11285, 1),
  (11279, 1),
  (11238, 1),
  (11298, 1),
  (11270, 1),
  (11255, 1),
  (11287, 1),
  (11281, 1),
  (11240, 1),
  (11300, 1),
  (11257, 1),
  (11289, 1),
  (11242, 1),
  (11302, 1),
  (11259, 1),
  (11291, 1),
  (11244, 1),
  (11304, 1),
  (11261, 1)],
 '26112012': [(4823, 1),
  (4874, 1),
  (4806, 1),
  (4787, 1),
  (4844, 1),
  (4825, 1),
  (4858, 1),
  (4876, 1),
  (4808, 1),
  (4789, 1),
  (4846, 1),
  (4827, 1),
  (4860, 1),
  (4878, 1),
  (4810, 1),
  (4791, 1),
  (4848, 1),
  (4829, 1),
  (4862, 1),
  (4880, 1),
  (4812, 1),
  (4793, 1),
  (4850, 1),
  (4831, 1),
  (4864, 1),
  (4882, 1),
  (4866, 1),
  (4816, 1),
  (4797, 1),
  (4835, 1),
  (4868, 1),
  (4818, 1),
  (4799, 1),
  (4837, 1)],
 '27012013': [(15579, 1),
  (15594, 1),
  (15622, 1),
  (15581, 1),
  (15609, 1),
  (15596, 1),
  (15624, 1),
  (15583, 1),
  (15598, 1),
  (15626, 1),
  (15585, 1),
  (15613, 1),
  (15600, 1),
  (15628, 1),
  (15587, 1),
  (15615, 1),
  (15602, 1),
  (15630, 1),
  (15589, 1),
  (15617, 1),
  (15604, 1),
  (15632, 1),
  (15591, 1),
  (15619, 1),
  (15606, 1)],
 '27112012': [(4856, 1),
  (4874, 1),
  (4787, 1),
  (4844, 1),
  (4825, 1),
  (4858, 1),
  (4876, 1),
  (4808, 1),
  (4789, 1),
  (4846, 1),
  (4827, 1),
  (4860, 1),
  (4878, 1),
  (4810, 1),
  (4791, 1),
  (4848, 1),
  (4829, 1),
  (4862, 1),
  (4880, 1),
  (4812, 1),
  (4793, 1),
  (4831, 1),
  (4864, 1),
  (4882, 1),
  (4795, 1),
  (4866, 1),
  (4816, 1),
  (4797, 1),
  (4835, 1),
  (4868, 1),
  (4818, 1),
  (4799, 1),
  (4837, 1)],
 '28012013': [(4644, 1),
  (4677, 1),
  (4695, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4650, 1),
  (4683, 1),
  (4701, 1),
  (4633, 1),
  (4614, 1),
  (4671, 1),
  (4652, 1),
  (4685, 1),
  (4703, 1),
  (4673, 1),
  (4654, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '28112012': [(4823, 1),
  (4856, 1),
  (4874, 1),
  (4806, 1),
  (4787, 1),
  (4858, 1),
  (4876, 1),
  (4808, 1),
  (4789, 1),
  (4846, 1),
  (4860, 1),
  (4878, 1),
  (4791, 1),
  (4829, 1),
  (4880, 1),
  (4812, 1),
  (4850, 1),
  (4831, 1),
  (4864, 1),
  (4866, 1),
  (4816, 1),
  (4797, 1),
  (4835, 1),
  (4868, 1),
  (4818, 1),
  (4837, 1)],
 '29012013': [(4627, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1),
  (4629, 1),
  (4610, 1),
  (4667, 1),
  (4648, 1),
  (4681, 1),
  (4699, 1),
  (4631, 1),
  (4612, 1),
  (4669, 1),
  (4650, 1),
  (4683, 1),
  (4633, 1),
  (4614, 1),
  (4671, 1),
  (4652, 1),
  (4685, 1),
  (4703, 1),
  (4635, 1),
  (4616, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '29112012': [(4823, 1),
  (4856, 1),
  (4787, 1),
  (4844, 1),
  (4825, 1),
  (4858, 1),
  (4876, 1),
  (4808, 1),
  (4846, 1),
  (4827, 1),
  (4860, 1),
  (4878, 1),
  (4810, 1),
  (4848, 1),
  (4829, 1),
  (4862, 1),
  (4880, 1),
  (4812, 1),
  (4793, 1),
  (4850, 1),
  (4864, 1),
  (4814, 1),
  (4852, 1),
  (4866, 1),
  (4816, 1),
  (4835, 1),
  (4868, 1),
  (4818, 1),
  (4799, 1),
  (4837, 1)],
 '30012013': [(4644, 1),
  (4677, 1),
  (4695, 1),
  (4627, 1),
  (4608, 1),
  (4629, 1),
  (4610, 1),
  (4635, 1),
  (4654, 1),
  (4687, 1),
  (4637, 1),
  (4618, 1),
  (4656, 1),
  (4689, 1),
  (4639, 1),
  (4620, 1),
  (4658, 1)],
 '30112012': [(4856, 1),
  (4787, 1),
  (4844, 1),
  (4858, 1),
  (4808, 1),
  (4789, 1),
  (4860, 1),
  (4878, 1),
  (4791, 1),
  (4848, 1),
  (4862, 1),
  (4880, 1),
  (4793, 1),
  (4831, 1),
  (4866, 1),
  (4816, 1),
  (4797, 1),
  (4835, 1),
  (4868, 1),
  (4818, 1),
  (4799, 1),
  (4837, 1)],
 '31012013': [(4677, 1),
  (4608, 1),
  (4665, 1),
  (4646, 1),
  (4679, 1),
  (4697, 1)]}

In [92]:
date_vjids_containing_start_stop_id={} # key:value is date:[vjids containing stop 226]
for item, row in df.iterrows():
    date=date_from_line(row)
    if date not in date_vjids_containing_start_stop_id:
        date_vjids_containing_start_stop_id[date]=[] # creating dictionary entry for date

for date in date_vjids_containing_start_stop_id: 
    for item, row in df.iterrows():
        if date_from_line(row)==date and row['VehicleJourneyID'] not in date_vjids_containing_start_stop_id[date] and row['StopID']==226:
            date_vjids_containing_start_stop_id[date].append(row['VehicleJourneyID'])
date_vjids_containing_start_stop_id

f = open ('date_vjids_containing_start_stop_id.txt', 'w')
f.write(str(date_vjids_containing_start_stop_id))
f.close()
g = open('date_vjids_containing_start_stop_id.txt', 'r')
g.read()

"{'06112012': [5461, 5494, 5444, 5425, 5482, 5463, 5496, 5514, 5446, 5427, 5484, 5465, 5498, 5516, 5486, 5467, 5500, 5518, 5450, 5488, 5469, 5502, 5452, 5433, 5490, 5471, 5504, 5454, 5435, 5473, 5506, 5456, 5437, 5475], '07112012': [5494, 5512, 5444, 5425, 5482, 5463, 5496, 5514, 5446, 5427, 5484, 5465, 5498, 5516, 5448, 5486, 5467, 5500, 5518, 5450, 5431, 5488, 5469, 5502, 5490, 5471, 5504, 5454, 5435, 5473, 5506, 5456, 5437, 5475], '08112012': [5461, 5494, 5512, 5444, 5425, 5482, 5463, 5496, 5514, 5446, 5427, 5484, 5465, 5498, 5516, 5448, 5486, 5467, 5500, 5518, 5450, 5431, 5488, 5469, 5502, 5520, 5452, 5490, 5504, 5454, 5435, 5473, 5506, 5456, 5437, 5475], '09112012': [5461, 5512, 5444, 5425, 5482, 5463, 5496, 5514, 5446, 5427, 5484, 5465, 5498, 5516, 5448, 5429, 5486, 5467, 5450, 5469, 5452, 5454, 5435, 5473, 5506, 5456, 5437, 5475], '10112012': [12138, 12164, 12123, 12155, 12140, 12166, 12125, 12185, 12157, 12127, 12176, 12170, 12129, 12189, 12161, 12146, 12172, 12131, 12191, 1214

In [93]:
# df_new=pd.DataFrame(columns=list(df)) # dataframe containing data from routes starting at stop 226, the known route starting point
# for item, row in df.iterrows():
#     if row['VehicleJourneyID'] in date_vjids_containing_start_stop_id[date_from_line(row)]:
#         df_new=df_new.append(row, ignore_index=True)
# print(df_new)

In [94]:
#this dict has every date in dataset as a key. The values for each key are the 
#VJIDs that operated on that date and their 
#frequency on that day. This will show when VJIDs were repeated on a given day.

VJ_frequency = {}
count = 1 
count2 = 0
for date in all_dates:
    if date not in VJ_frequency:
        VJ_frequency[date] = [] #[(row['VehicleJourneyID']), ]

for date in VJ_frequency:
    for item, row in dfStop.iterrows():
        #if (count2 % 100000):
            #print(count2)
        if date_from_line(row) == date and row['VehicleJourneyID'] not in VJ_frequency[date]:
            VJ_frequency[date].append((row['VehicleJourneyID'], count))
        elif date_from_line(row) == date and row['VehicleJourneyID'] in VJ_frequency[date]:
            VJ_frequency[date][indexOf(row['VehicleJourneyID'])+1]+=1
        #count2 +=1
                               
VJ_frequency            

{'01012013': [],
 '02012013': [],
 '03012013': [],
 '04012013': [],
 '05012013': [],
 '06012013': [],
 '06112012': [(5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5461, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5494, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5512, 1),
  (5444, 1),
  (5444, 1),
  

In [96]:
for key in VJ_frequency:
    print(key, VJ_frequency[key])

06112012 [(5461, 1), (5461, 1), (5461, 1), (5461, 1), (5461, 1), (5461, 1), (5461, 1), (5461, 1), (5461, 1), (5461, 1), (5461, 1), (5461, 1), (5461, 1), (5461, 1), (5461, 1), (5461, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5494, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5512, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5444, 1), (5425, 1), (5425, 1), 

In [65]:
dfStop = df.loc[df["StopID"]==226]
dfStop.head(4)

,Year,Month,Day,Hours,Minute,Second,WeekDay,Timestamp,JourneyPatternID,VehicleJourneyID,StopID,Temperature (C),Visibility (km),Wind Speed (km/h),Cumulative Time Taken
0,2012,11,6,8,59,2,1,1352192342,10001,5461,226,7,10,29.6,0
16,2012,11,6,9,21,8,1,1352193668,10001,5494,226,7,10,27.8,0
66,2012,11,6,10,0,9,1,1352196009,10001,5444,226,8,10,33.3,0
88,2012,11,6,10,20,18,1,1352197218,10001,5425,226,8,10,33.3,0


# create static timetables

In [66]:
VJWeekday=[]
VJSat=[]
VJSun=[]
VJUnique=[]

for item, row in dfStop.iterrows(): 
    if row['WeekDay'] >= 0:
        if row['WeekDay'] <= 4 and row['VehicleJourneyID'] not in VJWeekday:
            VJWeekday.append(row['VehicleJourneyID'])
    if row['WeekDay'] == 5:
        if row['VehicleJourneyID'] not in VJSat:
            VJSat.append(row['VehicleJourneyID'])
    if row['WeekDay'] == 6:
        if row['VehicleJourneyID'] not in VJSun:
            VJSun.append(row['VehicleJourneyID'])
print("M-F", VJWeekday, "\n", "\n", "Sat", VJSat, "\n", "\n", "Sun", VJSun)
print("M-F", len(set(VJWeekday)), "\n", "\n", "Sat", len(set(VJSat)), "\n", "\n", "Sun", len(set(VJSun)))

#for item, row in dfStop.iterrows():
for i in VJWeekday:
    if i not in VJUnique:
        VJUnique.append(i)
for i in VJSat:
    if i not in VJUnique:
        VJUnique.append(i)
for i in VJSun:
    if i not in VJUnique:
        VJUnique.append(i)
print("total VJUnique: ", len(VJUnique), len(set(VJUnique)))
print(VJUnique)


M-F [5461, 5494, 5444, 5425, 5482, 5463, 5496, 5514, 5446, 5427, 5484, 5465, 5498, 5516, 5486, 5467, 5500, 5518, 5450, 5488, 5469, 5502, 5452, 5433, 5490, 5471, 5504, 5454, 5435, 5473, 5506, 5456, 5437, 5475, 5512, 5448, 5431, 5520, 5429, 4784, 4817, 4835, 4748, 4805, 4786, 4819, 4837, 4769, 4750, 4807, 4821, 4771, 4752, 4809, 4790, 4823, 4841, 4754, 4811, 4825, 4775, 4756, 4794, 4827, 4777, 4796, 4829, 4779, 4760, 4798, 4839, 4813, 4758, 4767, 4788, 4773, 4843, 4874, 4806, 4787, 4844, 4858, 4876, 4808, 4789, 4846, 4860, 4878, 4810, 4791, 4848, 4862, 4880, 4812, 4793, 4850, 4831, 4864, 4882, 4866, 4816, 4797, 4868, 4818, 4799, 4856, 4795, 4814, 4852, 15594, 15622, 15581, 15609, 15596, 15624, 15583, 15611, 15598, 15626, 15585, 15613, 15600, 15628, 15587, 15615, 15602, 15630, 15589, 15617, 15604, 15632, 15591, 15619, 15606, 4644, 4677, 4608, 4665, 4646, 4679, 4629, 4610, 4667, 4648, 4681, 4699, 4631, 4612, 4669, 4683, 4633, 4614, 4671, 4652, 4685, 4703, 4635, 4616, 4673, 4654, 4687, 4637

In [67]:
VJWeekday=[]
VJMon, VJTue, VJWed, VJThur, VJFri = [], [], [], [], []

for item, row in dfStop.iterrows(): 
    if row['WeekDay'] == 0 and row['VehicleJourneyID'] not in VJMon:
            VJMon.append(row['VehicleJourneyID']) #, time_from_line(row), date_from_line[row]])
            
    if row['WeekDay'] == 1 and row['VehicleJourneyID'] not in VJTue:
            VJTue.append(row['VehicleJourneyID'])
            
    if row['WeekDay'] == 2 and row['VehicleJourneyID'] not in VJWed:
            VJWed.append(row['VehicleJourneyID'])
            
    if row['WeekDay'] == 3 and row['VehicleJourneyID'] not in VJThur:
            VJThur.append(row['VehicleJourneyID'])
            
    if row['WeekDay'] == 4 and row['VehicleJourneyID'] not in VJFri:
            VJFri.append(row['VehicleJourneyID'])
            
print(VJMon, "\n", "\n", VJTue, "\n", "\n", VJWed, "\n", "\n", VJThur, "\n", "\n", VJFri)
print("Mon", len(set(VJMon)), "\n", "\n", "Tue", len(set(VJTue)), "\n", "\n", "Wed", len(set(VJWed)), "\n", "\n", "Thur", len(set(VJThur)), "\n", "\n", "Fri", len(set(VJFri)), "\n", "\n")


for i in VJMon:
    if i not in VJWeekday:
        VJWeekday.append(i)
        
for i in VJTue:
    if i not in VJWeekday:
        VJWeekday.append(i)
        
for i in VJWed:
    if i not in VJWeekday:
        VJWeekday.append(i)
        
for i in VJThur:
    if i not in VJWeekday:
        VJWeekday.append(i)
        
for i in VJFri:
    if i not in VJWeekday:
        VJWeekday.append(i)
    
# #for item, row in dfStop.iterrows():
# for i in VJWeekday:
#     if i not in VJUnique:
#         VJUnique.append(i)
# for i in VJSat:
#     if i not in VJUnique:
#         VJUnique.append(i)
# for i in VJSun:
#     if i not in VJUnique:
#         VJUnique.append(i)
# print("total VJUnique: ", len(VJUnique), len(set(VJUnique)))
# print(VJUnique)


[5461, 5494, 5444, 5425, 5482, 5463, 5496, 5514, 5446, 5427, 5484, 5465, 5498, 5516, 5448, 5429, 5486, 5467, 5518, 5450, 5431, 5488, 5502, 5520, 5433, 5490, 5504, 5454, 5435, 5473, 5506, 5456, 5437, 5475, 4784, 4817, 4835, 4748, 4805, 4786, 4819, 4837, 4769, 4750, 4807, 4821, 4771, 4752, 4809, 4790, 4823, 4841, 4754, 4811, 4825, 4775, 4756, 4794, 4827, 4777, 4796, 4829, 4779, 4760, 4798, 4874, 4806, 4787, 4844, 4858, 4876, 4808, 4789, 4846, 4860, 4878, 4810, 4791, 4848, 4862, 4880, 4812, 4793, 4850, 4831, 4864, 4882, 4866, 4816, 4797, 4868, 4818, 4799, 4644, 4677, 4627, 4608, 4665, 4646, 4679, 4697, 4629, 4610, 4667, 4648, 4681, 4699, 4631, 4612, 4669, 4650, 4683, 4701, 4633, 4614, 4671, 4652, 4685, 4703, 4616, 4673, 4654, 4687, 4637, 4618, 4656, 4689, 4639, 4620, 4658, 4695, 4635] 
 
 [5461, 5494, 5444, 5425, 5482, 5463, 5496, 5514, 5446, 5427, 5484, 5465, 5498, 5516, 5486, 5467, 5500, 5518, 5450, 5488, 5469, 5502, 5452, 5433, 5490, 5471, 5504, 5454, 5435, 5473, 5506, 5456, 5437, 5475

unique vjids running on each week category:
M-F:173, Sat:102, Sun:92

In [68]:
print(VJWeekday[0:10])

[5461, 5494, 5444, 5425, 5482, 5463, 5496, 5514, 5446, 5427]


In [69]:
#Sat has no vjids which are present in other categories,
#Weekday has 25 identical in Sun
countSat = 0
countSun = 0
countW = 0
WeekSun = []
for i in VJSun:
    if i in VJSat:
        countSat+=1
    elif i in VJWeekday:
        countW+=1
        WeekSun.append(i)
print("Saturday VJIDs that run on other days:", countSat)
print("Weekday VJIDs that run on Sundays:", countW)
print(WeekSun)
print(len(WeekSun))

Saturday VJIDs that run on other days: 0
Weekday VJIDs that run on Sundays: 25
[15594, 15622, 15581, 15609, 15596, 15624, 15583, 15611, 15598, 15626, 15585, 15613, 15600, 15628, 15587, 15615, 15602, 15630, 15589, 15617, 15604, 15632, 15591, 15619, 15606]
25


Time categories

In [70]:
morningP = [7, 8, 9, 10, 11]
midday = [12, 13, 14, 15]
eveningP = [16, 17, 18, 19]
night = [20, 21, 22, 23]

In [71]:
print("M-F", len(VJWeekday), "\n", "Sat", len(VJSat), "\n", "Sun", len(VJSun))

M-F 173 
 Sat 102 
 Sun 92


In [72]:
morningTrips = []
middayTrips = []
eveningTrips = []
nightTrips = []

In [73]:
#only Sunday data
dfSun = dfStop.loc[dfStop['WeekDay']==6]
dfSun.head(10)

,Year,Month,Day,Hours,Minute,Second,WeekDay,Timestamp,JourneyPatternID,VehicleJourneyID,StopID,Temperature (C),Visibility (km),Wind Speed (km/h),Cumulative Time Taken
3592,2012,11,11,11,0,4,6,1352631604,10001,16367,226,6,10,16.7,0
3602,2012,11,11,12,0,7,6,1352635207,10001,16354,226,7,40,21.6,0
3623,2012,11,11,12,29,47,6,1352636987,10001,16382,226,7,10,22.2,0
3638,2012,11,11,13,0,42,6,1352638842,10001,16369,226,8,10,16.7,0
3656,2012,11,11,14,0,25,6,1352642425,10001,16356,226,8,10,18.5,0
3674,2012,11,11,14,30,7,6,1352644207,10001,16384,226,8,10,16.7,0
3689,2012,11,11,15,0,4,6,1352646004,10001,16371,226,8,10,13,0
3711,2012,11,11,15,59,45,6,1352649585,10001,16358,226,9,10,13,0
3726,2012,11,11,16,30,24,6,1352651424,10001,16386,226,8,10,11.1,0
3744,2012,11,11,17,0,44,6,1352653244,10001,16373,226,7,10,13,0


In [74]:
#Sunday in November
dfSunN = dfSun.loc[dfStop['Month']==11]

for col in dfSunN:
    print([col], dfSunN[col].unique())

['Year'] [2012]
['Month'] [11]
['Day'] [11 18 25]
['Hours'] [11 12 13 14 15 16 17 18 20 22 10 19 21 23]
['Minute'] [ 0 29 30 59 32 31 58  1 28 33  2  6]
['Second'] [ 4  7 47 42 25 45 24 44 10 31  8 46 43  3 27 49 26 23 30 48 12  5  1  6 22]
['WeekDay'] [6]
['Timestamp'] [1352631604 1352635207 1352636987 1352638842 1352642425 1352644207
 1352646004 1352649585 1352651424 1352653244 1352656782 1352658610
 1352660385 1352664031 1352665928 1352667586 1352671210 1352673044
 1352674783 1353234624 1353236403 1353238227 1353240045 1353241904
 1353243589 1353245507 1353247226 1353249043 1353250785 1353252623
 1353254404 1353256243 1353257989 1353259787 1353261630 1353263568
 1353265129 1353267045 1353268812 1353270588 1353272448 1353274205
 1353276003 1353277785 1353279631 1353841304 1353843001 1353844788
 1353846583 1353848406 1353850208 1353853704 1353855552 1353857406
 1353861066 1353862827 1353866405 1353872002 1353873725 1353875386
 1353877303 1353879188 1353880863 1353882607 1353884787]
['

In [75]:
#sunday 11th, nov
dfSunOneWeekN = dfSunN.loc[(dfStop['Day'] == 11)] 
print(dfSunOneWeekN['Day'].unique())

[11]


In [76]:
#dfSunOneWeekN.unique()
for col in dfSunOneWeekN:
    print([col], dfSunOneWeekN[col].unique())

['Year'] [2012]
['Month'] [11]
['Day'] [11]
['Hours'] [11 12 13 14 15 16 17 18 20 22]
['Minute'] [ 0 29 30 59 32]
['Second'] [ 4  7 47 42 25 45 24 44 10 31  8 46 43]
['WeekDay'] [6]
['Timestamp'] [1352631604 1352635207 1352636987 1352638842 1352642425 1352644207
 1352646004 1352649585 1352651424 1352653244 1352656782 1352658610
 1352660385 1352664031 1352665928 1352667586 1352671210 1352673044
 1352674783]
['JourneyPatternID'] [10001]
['VehicleJourneyID'] [16367 16354 16382 16369 16356 16384 16371 16358 16386 16373 16360 16388
 16375 16362 16390 16377 16364 16392 16379]
['StopID'] [226]
['Temperature (C)'] [6 7 8 9]
['Visibility (km)'] ['10' '40']
['Wind Speed (km/h)'] ['16.7' '21.6' '22.2' '18.5' '13' '11.1' '9.3' '14.8' '7.4']
['Cumulative Time Taken'] [0]


In [77]:
dfSunOneWeekN.head(4)

,Year,Month,Day,Hours,Minute,Second,WeekDay,Timestamp,JourneyPatternID,VehicleJourneyID,StopID,Temperature (C),Visibility (km),Wind Speed (km/h),Cumulative Time Taken
3592,2012,11,11,11,0,4,6,1352631604,10001,16367,226,6,10,16.7,0
3602,2012,11,11,12,0,7,6,1352635207,10001,16354,226,7,40,21.6,0
3623,2012,11,11,12,29,47,6,1352636987,10001,16382,226,7,10,22.2,0
3638,2012,11,11,13,0,42,6,1352638842,10001,16369,226,8,10,16.7,0


In [78]:
#for sunday
#grouping vjid by hour of day into 4 subcategories; morning, middday, evening, night
#for i in VJUnique:
for item, row in dfSunOneWeekN.iterrows(): #sunday, 11th, november
#     if row['VehicleJourneyID'] in VJUnique: 
        if row['Hours'] in morningP and row['VehicleJourneyID'] not in morningTrips:
            morningTrips.append(row['VehicleJourneyID'])
        elif row['Hours'] in midday and row['VehicleJourneyID'] not in middayTrips:
            middayTrips.append(row['VehicleJourneyID'])
        elif row['Hours'] in eveningP and row['VehicleJourneyID'] not in eveningTrips:
            eveningTrips.append(row['VehicleJourneyID'])
        elif row['Hours'] in night and row['VehicleJourneyID'] not in nightTrips:
            nightTrips.append(row['VehicleJourneyID'])
        else: 
            continue
print(len(morningTrips), len(middayTrips), len(eveningTrips), len(nightTrips))
print(len(set(morningTrips)), len(set(middayTrips)), len(set(eveningTrips)), len(set(nightTrips)))
print(morningTrips, "\n", middayTrips, "\n", eveningTrips, "\n", nightTrips)

1 7 5 6
1 7 5 6
[16367] 
 [16354, 16382, 16369, 16356, 16384, 16371, 16358] 
 [16386, 16373, 16360, 16388, 16375] 
 [16362, 16390, 16377, 16364, 16392, 16379]


In [79]:
# import operator
# x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
# sorted_x = sorted(x.items(), key=operator.itemgetter(1))
# sorted_x

In [80]:
print("Unique Sunday trips")
print(morningTrips, "\n", len(set(morningTrips)))
print(middayTrips, "\n", len(set(middayTrips)))
print(eveningTrips, "\n", len(set(eveningTrips)))
print(nightTrips, len(set(nightTrips)))
count = int(len(set(morningTrips))) + int(len(set(middayTrips))) + int(len(set(eveningTrips))) + int(len(set(nightTrips)))
#print("\n", sum(sum(int(morningTrips), int(middayTrips)), sum(int(eveningTrips), int(nightTrips))))
print(count)

Unique Sunday trips
[16367] 
 1
[16354, 16382, 16369, 16356, 16384, 16371, 16358] 
 7
[16386, 16373, 16360, 16388, 16375] 
 5
[16362, 16390, 16377, 16364, 16392, 16379] 6
19


In [81]:
list = [morningTrips, middayTrips, eveningTrips, nightTrips]

dict1 = {}
for i in list:
    for j in i:
        if j not in dict1:
            dict1[j] = 1
        elif j in dict1:
            dict1[j] +=1
    print(dict1, "\n")


{16367: 1} 

{16367: 1, 16354: 1, 16382: 1, 16369: 1, 16356: 1, 16384: 1, 16371: 1, 16358: 1} 

{16367: 1, 16354: 1, 16382: 1, 16369: 1, 16356: 1, 16384: 1, 16371: 1, 16358: 1, 16386: 1, 16373: 1, 16360: 1, 16388: 1, 16375: 1} 

{16367: 1, 16354: 1, 16382: 1, 16369: 1, 16356: 1, 16384: 1, 16371: 1, 16358: 1, 16386: 1, 16373: 1, 16360: 1, 16388: 1, 16375: 1, 16362: 1, 16390: 1, 16377: 1, 16364: 1, 16392: 1, 16379: 1} 



In [82]:
allSun = [] 
for i in morningTrips:
    allSun.append(i)
for i in middayTrips:
    allSun.append(i)
for i in eveningTrips:
    allSun.append(i)
for i in nightTrips:
    allSun.append(i)
print(len(set(allSun)), allSun)

19 [16367, 16354, 16382, 16369, 16356, 16384, 16371, 16358, 16386, 16373, 16360, 16388, 16375, 16362, 16390, 16377, 16364, 16392, 16379]


In [83]:
#dict of sunday start times. Key = vjid: value = start time
sundayTT = {}
for item, row in dfStop.iterrows(): #only using trips starting at stop 226
    if row['WeekDay'] == 6:
        time=time_from_line(row)
        date=date_from_line(row)
        if row['VehicleJourneyID'] in VJSun and row['VehicleJourneyID'] not in sundayTT:
            sundayTT[row['VehicleJourneyID']] = [(date,time)]
        #elif row['VehicleJourneyID'] in VJSun and row['VehicleJourneyID'] in sundayTT:
        elif row['VehicleJourneyID'] in sundayTT:
            sundayTT[row['VehicleJourneyID']].append((date, time))

print("sundayTT", "\n", sundayTT, "\n", "num of keys:", len(sundayTT[15596]))
for i, j in sundayTT[15596]:
    print(i,j)
    

sundayTT 
 {16367: [('11112012', '110004')], 16354: [('11112012', '120007')], 16382: [('11112012', '122947')], 16369: [('11112012', '130042')], 16356: [('11112012', '140025')], 16384: [('11112012', '143007')], 16371: [('11112012', '150004')], 16358: [('11112012', '155945')], 16386: [('11112012', '163024')], 16373: [('11112012', '170044')], 16360: [('11112012', '175942')], 16388: [('11112012', '183010')], 16375: [('11112012', '185945')], 16362: [('11112012', '200031')], 16390: [('11112012', '203208')], 16377: [('11112012', '205946')], 16364: [('11112012', '220010')], 16392: [('11112012', '223044')], 16379: [('11112012', '225943')], 15672: [('18112012', '103024')], 15659: [('18112012', '110003')], 15687: [('18112012', '113027')], 15646: [('18112012', '120045')], 15674: [('18112012', '123144')], 15661: [('18112012', '125949')], 15689: [('18112012', '133147')], 15648: [('18112012', '140026')], 15676: [('18112012', '143043')], 15663: [('18112012', '145945')], 15691: [('18112012', '153023')]

In [38]:
    print(key, value)

16367 [('11112012', '110004')]
16354 [('11112012', '120007')]
16382 [('11112012', '122947')]
16369 [('11112012', '130042')]
16356 [('11112012', '140025')]
16384 [('11112012', '143007')]
16371 [('11112012', '150004')]
16358 [('11112012', '155945')]
16386 [('11112012', '163024')]
16373 [('11112012', '170044')]
16360 [('11112012', '175942')]
16388 [('11112012', '183010')]
16375 [('11112012', '185945')]
16362 [('11112012', '200031')]
16390 [('11112012', '203208')]
16377 [('11112012', '205946')]
16364 [('11112012', '220010')]
16392 [('11112012', '223044')]
16379 [('11112012', '225943')]
15672 [('18112012', '103024')]
15659 [('18112012', '110003')]
15687 [('18112012', '113027')]
15646 [('18112012', '120045')]
15674 [('18112012', '123144')]
15661 [('18112012', '125949')]
15689 [('18112012', '133147')]
15648 [('18112012', '140026')]
15676 [('18112012', '143043')]
15663 [('18112012', '145945')]
15691 [('18112012', '153023')]
15650 [('18112012', '160004')]
15678 [('18112012', '163043')]
15665 [(

In [39]:
# for key, value in sundayTT.items():
#     print(key, len(value))

In [40]:
#get list of vjids with multiple trips start times
#get average run time
multiple = []
for key, value in sundayTT.items():
    if len(value) > 1:
        multiple.append(key)
print("multiple", multiple)

multiple [15594, 15622, 15581, 15609, 15596, 15624, 15583, 15611, 15598, 15626, 15585, 15613, 15600, 15628, 15587, 15615, 15602, 15630, 15589, 15617, 15604, 15632, 15591, 15619, 15606]


In [41]:
# for key, value in sundayTT.items():
#     if len(value) > 1:
#         print(key)
#         print("entire", value)
#         print("0-1", value[0:1])
#         print("1-4", value[0:4])

In [42]:
#get average of times of bus run times
def convert_to_seconds(time):
    hours = str(time)[0:2]
    minutes = str(time)[2:4]
    seconds = str(time)[4:6]
    hSec = int(hours) * 60 * 60
    mSec = int(minutes) * 60
    total = int(hSec) + int(mSec) + int(seconds)
    return total
    #print(hours, minutes, seconds)
    #print(total)
#convert_seconds(124903)

In [43]:
keySum = 0
for key, value in sundayTT.items():
    if key in multiple:
        for i in range(0, len(value)):
            keySum+=(convert_to_seconds(value[i][1]))
        keySum = keySum/len(value)
        print(key,  len(value), value, "keySum", keySum)
            

15594 4 [('06012013', '110007'), ('13012013', '110046'), ('20012013', '110028'), ('27012013', '105947')] keySum 39617.0
15622 4 [('06012013', '113004'), ('13012013', '112949'), ('20012013', '112951'), ('27012013', '113004')] keySum 51301.25
15581 4 [('06012013', '120028'), ('13012013', '120104'), ('20012013', '120009'), ('27012013', '120206')] keySum 56082.0625
15609 4 [('06012013', '123007'), ('13012013', '123102'), ('20012013', '123003'), ('27012013', '123023')] keySum 59044.265625
15596 4 [('06012013', '130031'), ('13012013', '130042'), ('20012013', '125942'), ('27012013', '125922')] keySum 61565.31640625
15624 4 [('06012013', '132943'), ('13012013', '133129'), ('20012013', '132949'), ('27012013', '133008')] keySum 64008.5791015625
15583 4 [('06012013', '135951'), ('13012013', '135944'), ('20012013', '140024'), ('27012013', '135944')] keySum 66397.89477539062
15611 2 [('06012013', '143006'), ('20012013', '143003')] keySum 85403.44738769531
15598 4 [('06012013', '150009'), ('13012013

In [44]:
# #convert seconds to hhmmss
# def convert_to_HHMMSS(seconds): #already divided by len(value), average
#     minutes = seconds / 60
#     secondsR = int(seconds % 60)
#     if int(secondsR) < 10:
#         secondsR = "0"+str(secondsR)
#     print("seconds", secondsR)
#     print("SECONDS:", ((seconds - secondsR)//60))
#     print("MINUTES: ", minutes, int(minutes%60), int(minutes//60)*60, int((minutes - (minutes//60)) /60))
    
#     hours = int((minutes //60)) #- (minutes%60)
#     if int(hours) < 10:
#         hours = "0"+str(hours)
#     print("hours", hours)
        
#     minutesR = int(minutes % 60)
#     print(type(minutesR))
#     if int(minutes) < 10 : #or len(str(minutes)) < 2:
#         minutes = "0"+str(minutesR)
#     print("minutes", minutesR)
    
    
#     time_correct = str(hours) + str(int(minutesR)) + str(secondsR)
#     return time_correct
        
#     #print(hours, minutes, secondsR)
    
# print(convert_to_HHMMSS(39617.0))
# #print(convert_to_HHMMSS(107275.82872207))
# #print(convert_to_HHMMSS(92336.782))
# #print(convert_to_HHMMSS(51301.25))

In [45]:
# for key, value in sundayTT.items():
#     print(key, value)

dict of sunday key is vjid, value is time it starts

In [46]:
# print(list(df))

In [47]:
# #make df of all vj from stopid 226
# cols = list(df)
# new_df2 = pd.DataFrame(columns=list(df))
# for item, row in dfStop.iterrows():
#     if row['VehicleJourneyID']==5461 and row['StopID']==226:
#         #print("first")
#         #if row['StopID']==226: #row['VehicleJourneyID']==5444 &
#             #print("second")
#             new_df2 = new_df2.append(row, ignore_index=True)
# print(new_df2)

In [48]:
# #date=date_from_line(row)
# for item, row in new_df2.iterrows():
#     #print(row['Day'], row['Month'], date_from_line(row))
#     row["Date"] = date_from_line(row) #stopped here
# print(new_df2.head(5))


In [49]:
# dfVJ = df.loc[df["Vehicle Journey ID"] == 5425] #date needed to make VJID unique for a stopID
# dfVJ_06 = dfVJ.loc[dfVJ["Date"] == "06/11/2012"]
# dfVJ_06.shape

In [50]:
# dfVJ_06.head(1)

In [51]:
# dfVJ_06[['Stop ID', 'Cumulative Time Taken']].corr()

In [52]:
# dfVJ_06.plot(kind='bar', x='Stop ID', y='Cumulative Time Taken')

In [53]:
# for col_name in df[categ_cols]:
#     print(col_name, "\n", df[col_name].unique(), "\n", "number of unique values:", len(df[col_name].unique()), "\n", df[col_name].value_counts(), "\n")

In [54]:
# df.head(1)

look at CTT for each StopID for each vjid

Groups: M-F, S, Su. All trips happen during the day, from ~9 to evening time.

In [55]:
# M_F_List = []
# S_List = []
# Su_List = []
# #print(df.columns)
# for item, row  in df.iteritems():
#     if (row['WeekDay'] == 1): # == 'True': #| 
#         M_F_List.append(row["Vehicle Journey ID"])

Feature Choice: CTT

In [56]:
x = dfSun
for col_name in x:
    print(col_name, "\n", x[col_name].unique(), "\n", "number of unique values:", len(x[col_name].unique()), "\n", x[col_name].value_counts(), "\n")


Year 
 [2012 2013] 
 number of unique values: 2 
 2013    98
2012    65
Name: Year, dtype: int64 

Month 
 [11  1] 
 number of unique values: 2 
 1     98
11    65
Name: Month, dtype: int64 

Day 
 [11 18 25  6 13 20 27] 
 number of unique values: 7 
 18    26
27    25
20    25
6     25
13    23
25    20
11    19
Name: Day, dtype: int64 

Hours 
 [11 12 13 14 15 16 17 18 20 22 10 19 21 23] 
 number of unique values: 14 
 12    16
22    15
20    15
18    14
16    13
13    13
11    13
21    12
15    12
14    11
17    10
19     9
23     6
10     4
Name: Hours, dtype: int64 

Minute 
 [ 0 29 30 59 32 31 58  1 28 33  2  6] 
 number of unique values: 12 
 0     57
30    41
59    19
29    14
31    11
1      7
33     4
32     3
2      3
58     2
28     1
6      1
Name: Minute, dtype: int64 

Second 
 [ 4  7 47 42 25 45 24 44 10 31  8 46 43  3 27 49 26 23 30 48 12  5  1  6 22
 28 51  9  2 50 29  0 32] 
 number of unique values: 33 
 4     11
43     9
3      8
6      8
44     8
24     8
49     7

In [57]:
dfSun.head(2)

,Year,Month,Day,Hours,Minute,Second,WeekDay,Timestamp,JourneyPatternID,VehicleJourneyID,StopID,Temperature (C),Visibility (km),Wind Speed (km/h),Cumulative Time Taken
3592,2012,11,11,11,0,4,6,1352631604,10001,16367,226,6,10,16.7,0
3602,2012,11,11,12,0,7,6,1352635207,10001,16354,226,7,40,21.6,0
